In [9]:
import pprint
import numpy as np
import pandas as pd
import scipy.linalg as sl
from sklearn.linear_model import LinearRegression
from scipy.linalg import cho_factor, cho_solve

## Problem 1

#### 1.1 Read the data and include an intercept in your model

In [10]:
# Read the data by using pandas
longley = pd.read_csv("longley.dat", header=None, delim_whitespace=True)

In [72]:
# Rename the column 
longley.rename(columns={0:"number of people employed", 1:"GNP implicit price deflator", 2: "GNP", 3:"number of unemployed",
4: "number of people in the armed forces", 5: "noninst. pop", 6: "year"}, inplace=True)

print(longley.head())
print(longley.tail())


   number of people employed  GNP implicit price deflator     GNP  \
0                      60323                         83.0  234289   
1                      61122                         88.5  259426   
2                      60171                         88.2  258054   
3                      61187                         89.5  284599   
4                      63221                         96.2  328975   

   number of unemployed  number of people in the armed forces  noninst. pop  \
0                  2356                                  1590        107608   
1                  2325                                  1456        108632   
2                  3682                                  1616        109773   
3                  3351                                  1650        110929   
4                  2099                                  3099        112075   

   year  
0  1947  
1  1948  
2  1949  
3  1950  
4  1951  
    number of people employed  GNP implicit price 

In [12]:
lm = LinearRegression()

model = lm.fit(longley.iloc[:,0:1], longley.iloc[:,1:])

model.coef_
model.intercept_

print(np.array_str(model.intercept_, precision=1, suppress_small=True))

[     -93.2 -1430482.3    -5539.9    -3312.2    -6824.      1868.5]


### 1.2

#### GE/LU decompostion

In [41]:
A = np.array(longley.iloc[:,1:])
b = np.array(longley.iloc[:,0:1])

P, L, U = sl.lu(A)


# print(np.array_str(L, precision=1, suppress_small=True))
# print(np.array_str(U, precision=1, suppress_small=True))

print(L.shape)
print(U.shape)
print(A.shape)
print(b.shape)

(16, 6)
(6, 6)
(16, 6)
(16, 1)


In [47]:
print(np.array_str(L, precision=1, suppress_small=True))

[[ 1.   0.   0.   0.   0.   0. ]
 [ 0.8  1.   0.   0.   0.   0. ]
 [ 0.8  1.   1.   0.   0.   0. ]
 [ 0.8  0.7 -0.7  1.   0.   0. ]
 [ 0.9  0.5  0.9  0.1  1.   0. ]
 [ 1.   0.3  0.1  0.   0.7  1. ]
 [ 0.8  0.7 -0.8  0.9 -0.2 -0.4]
 [ 0.9  0.7  0.5  0.7  0.1 -0.3]
 [ 0.9  0.5 -0.1  0.5 -0.1  1. ]
 [ 0.9  0.5 -0.3  0.4  0.2  0.8]
 [ 0.9  0.4 -0.3  0.3  0.5  0.4]
 [ 0.8  0.8 -0.5  0.8  0.1  0.3]
 [ 0.8  0.9  0.7 -0.  -0.2  0.7]
 [ 1.   0.2  0.1 -0.1  0.6  0.6]
 [ 1.   0.2  0.7 -0.1  0.5 -0.1]
 [ 0.7  1.   0.2  0.1 -0.7  0.1]]


In [64]:
def solve(L, b):
    v = [0 for i in range(len(L))]
    for i in range(len(L)):
        v[i] = (b[i] - np.dot(v, L[i]))/float(L[i][i])

In [73]:
v = np.array([0 for i in range(len(L))])
v.shape

(16,)

In [74]:
def forsolve(L,b):
    x = np.zeros_like(b)
    for i in range(x.shape[0]):
        sum=0.
        for j in range(i):
            sum += L[i,j]*x[j]
        x[i] = (b[i]-sum)/L[i,i]
    return x.reshape(-1,1)

In [77]:
x = np.zeros_like(b)
x

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]], dtype=int64)

In [52]:
solve(L,b)

ValueError: shapes (16,) and (6,) not aligned: 16 (dim 0) != 6 (dim 0)

#### QR decompostion

In [79]:
q, r = sl.qr(A)

# print(np.array_str(q, precision=1, suppress_small=True))
# print(np.array_str(r, precision=1, suppress_small=True))

q

array([[-0.20300008, -0.38280381, -0.01764997, -0.260406  ,  0.40314384,
         0.07765528, -0.39259085, -0.19530657, -0.17072541, -0.13604157,
        -0.1116941 ,  0.08949862,  0.05415203, -0.09854322, -0.23077236,
        -0.49926218],
       [-0.21645189, -0.36577761,  0.0588566 , -0.47913758, -0.22228952,
         0.31773649, -0.12700722,  0.23729603,  0.33209354,  0.10702125,
        -0.0662363 ,  0.17717319,  0.20849959,  0.14209883,  0.26369644,
         0.26979959],
       [-0.21571816, -0.36671024, -0.41820355,  0.02390594,  0.04202782,
         0.04941875,  0.25173702, -0.43497872, -0.01077471,  0.16606296,
         0.24834315, -0.42562883,  0.06464633,  0.12637096, -0.15793712,
         0.26667749],
       [-0.21889768, -0.27181451, -0.26063351, -0.1170797 ,  0.19824471,
        -0.35977146,  0.40584285,  0.42674714, -0.25748465, -0.17945066,
         0.0091855 ,  0.22188287, -0.28656954, -0.12922621,  0.16869929,
         0.07685672],
       [-0.23528443, -0.19040934,  0